In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, r2_score
from sklearn.decomposition import PCA

In [ ]:
# --- 1. LOAD DATA ---
df = pd.read_csv('radial_features_labels.csv')
print(df.head())

In [ ]:
# --- 2. VISUALIZATION: FEATURE DISTRIBUTIONS ---
plt.figure(figsize=(10,6))
for i, col in enumerate(df.columns[:-1]):
    plt.subplot(2,3,i+1)
    sns.histplot(df[col], kde=True)
    plt.title(f'Distribution: {col}')
plt.tight_layout()
plt.show()

In [ ]:
# --- 3. PAIRPLOT: FEATURE RELATIONSHIPS ---
sns.pairplot(df, diag_kind='kde')
plt.suptitle('Pairplot of Features and cfPWV', y=1.02)
plt.show()


In [ ]:
# --- 4. HEATMAP: FEATURE CORRELATIONS ---
plt.figure(figsize=(8,6))
corr = df.corr()
sns.heatmap(corr, annot=True, fmt=".2f", cmap='coolwarm')
plt.title('Correlation Matrix')
plt.show()

In [ ]:
# --- 5. ML WORKFLOW ---

# Features and labels
X = df.drop('cfPWV', axis=1)
y = df['cfPWV']

# Train/Test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Random Forest Regressor
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Predictions
y_pred = model.predict(X_test)

print(f'R2 score: {r2_score(y_test, y_pred):.3f}')
print(f'Mean Absolute Error: {mean_absolute_error(y_test, y_pred):.3f}')

In [ ]:
# --- 6. FEATURE IMPORTANCE ---
plt.figure(figsize=(8,4))
feat_imp = pd.Series(model.feature_importances_, index=X.columns)
feat_imp.sort_values().plot(kind='barh')
plt.title('Random Forest Feature Importances')
plt.xlabel('Importance')
plt.tight_layout()
plt.show()

In [ ]:
# --- 7. PREDICTED VS ACTUAL PLOT ---
plt.figure(figsize=(6,6))
plt.scatter(y_test, y_pred, alpha=0.5)
plt.plot([y.min(), y.max()], [y.min(), y.max()], 'r--', lw=2)
plt.xlabel('Actual cfPWV')
plt.ylabel('Predicted cfPWV')
plt.title('Predicted vs Actual cfPWV')
plt.tight_layout()
plt.show()

In [ ]:
# --- 8. PCA VISUALIZATION (OPTIONAL) ---
# If you have waveforms for each subject (e.g., exported from MATLAB)
# Assume you have waveform_data.csv: rows=subjects, cols=time points

try:
    wf = pd.read_csv('waveform_data.csv')  # Not required if you only use features
    pca = PCA(n_components=2)
    wf_pca = pca.fit_transform(wf)
    plt.figure(figsize=(8,6))
    plt.scatter(wf_pca[:,0], wf_pca[:,1], c=df['cfPWV'], cmap='plasma', alpha=0.7)
    plt.colorbar(label='cfPWV')
    plt.xlabel('PC1'); plt.ylabel('PC2')
    plt.title('PCA of Radial Waveforms Colored by cfPWV')
    plt.tight_layout()
    plt.show()
except Exception as e:
    print("Waveform PCA skipped (export waveform_data.csv if you want this plot)")

In [ ]:
# --- 9. ADVANCED: CROSS-VALIDATION SCORE ---
cv_scores = cross_val_score(model, X, y, cv=5, scoring='neg_mean_absolute_error')
print(f"Cross-validated MAE: {np.mean(-cv_scores):.3f} ± {np.std(cv_scores):.3f}")

# --- 10. OPTIONAL: PARTIAL DEPENDENCE PLOTS (Requires sklearn>=0.24) ---
try:
    from sklearn.inspection import PartialDependenceDisplay
    fig, ax = plt.subplots(figsize=(12, 6))
    PartialDependenceDisplay.from_estimator(
        model, X, features=range(X.shape[1]), feature_names=X.columns, ax=ax
    )
    plt.tight_layout()
    plt.show()
except ImportError:
    print("PartialDependenceDisplay not available in your sklearn version.")
